Section: Securing Federated Learning

Project: Federated Learning with a Trusted Aggregator

In [56]:
!pip install syft

In [57]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0816 19:04:23.512202 140471712753536 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [59]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice,bob])

W0816 19:04:23.545953 140471712753536 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0816 19:04:23.549605 140471712753536 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0816 19:04:23.551928 140471712753536 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0816 19:04:23.553277 140471712753536 base.py:646] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0816 19:04:23.556054 140471712753536 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0816 19:04:23.558022 140471712753536 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A toy dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Initialize A toy model
model = nn.Linear(2,1)

In [64]:
for round_iter in range(10):
  
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

  for i in range(10):

    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss

    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

  alices_model.move(secure_worker)
  bobs_model.move(secure_worker)

  with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
  secure_worker.clear_objects()
  
  print("Bob:" + str(bobs_loss) + "Alice:" + str(alices_loss))

Bob:tensor(4.7667e-05)Alice:tensor(0.)
Bob:tensor(2.4340e-05)Alice:tensor(0.)
Bob:tensor(1.2630e-05)Alice:tensor(0.)
Bob:tensor(6.6061e-06)Alice:tensor(0.)
Bob:tensor(3.4684e-06)Alice:tensor(0.)
Bob:tensor(1.8244e-06)Alice:tensor(0.)
Bob:tensor(9.6046e-07)Alice:tensor(0.)
Bob:tensor(5.0585e-07)Alice:tensor(0.)
Bob:tensor(2.6647e-07)Alice:tensor(0.)
Bob:tensor(1.4038e-07)Alice:tensor(0.)


Project: Build Methods for Encrypt, Decryptm, and Add

In [0]:
import random

In [0]:
Q = 234249863958762043895

In [0]:
def encrypt(x, n_shares=3):
  
  shares = list()
  
  for i in range(n_shares - 1):
    shares.append(random.randint(0,Q))
    
  final_share = Q - (sum(shares) % Q) + x
  
  shares.append(final_share)
  
  return tuple(shares)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [0]:
def add(a, b):
  
  c = list()
  
  assert(len(a) == len(b))
  
  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)
    
  return tuple(c)

In [70]:
decrypt(add(encrypt(5), encrypt(10)))

15

Lesson: Intro to Fixed Precision Encoding

In [0]:
BASE = 10
PRECISION = 4
Q = 2343564675676743345

In [0]:
def encode(x_dec):
  return int(x_dec * (BASE ** PRECISION)) % Q

In [0]:
def decode(x_fp):
  return (x_fp if x_fp <= Q/2 else x_fp - Q) / BASE ** PRECISION

In [79]:
encode(-0.5 - 0.5)

2343564675676733345

In [78]:
decode(2343564675676738345)

-0.5

Lesson: Secret Sharing + Fixed Precision in PySyft

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [82]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [85]:
x = x.send(bob, alice, secure_worker)
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:38839409845 -> bob:85557518568]
	-> [PointerTensor | me:7990357480 -> alice:27006599859]
	-> [PointerTensor | me:97924458133 -> secure_worker:47269200841]

In [86]:
bob._objects

{9409332212: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 20662629615: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 62584872626: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 70689600495: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 85557518568: (Wrapper)>[AdditiveSharingTensor]
 	-> tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499])
 	-> [PointerTensor | bob:38842800472 -> alice:67722860482]
 	-> [PointerTensor | bob:81120625400 -> secure_worker:22843969630]
 	*crypto provider: me*}

In [0]:
y = x + x

In [88]:
bob._objects

{9409332212: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 20662629615: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  319637444771099499]),
 30004191026: tensor([ 551747413404636730, 4565340713322011448, 3016499613353362258,
         3909889504980186798,  639274889542198998]),
 53768740351: tensor([ 551747413404636730, 4565340713322011448, 3016499613353362258,
         3909889504980186798,  639274889542198998]),
 61707577782: (Wrapper)>[AdditiveSharingTensor]
 	-> tensor([ 551747413404636730, 4565340713322011448, 3016499613353362258,
         3909889504980186798,  639274889542198998])
 	-> [PointerTensor | bob:31716758775 -> alice:59855583100]
 	-> [PointerTensor | bob:46315943438 -> secure_worker:61191797929]
 	*crypto provider: me*,
 62584872626: tensor([ 275873706702318365, 2282670356661005724, 1508249806676681129,
         1954944752490093399,  3196

In [89]:
y

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:18206243463 -> bob:61707577782]
	-> [PointerTensor | me:86541862069 -> alice:39175235206]
	-> [PointerTensor | me:48176507715 -> secure_worker:95932899999]

In [90]:
y.get()

[(Wrapper)>[AdditiveSharingTensor]
 	-> tensor([ 551747413404636730, 4565340713322011448, 3016499613353362258,
         3909889504980186798,  639274889542198998])
 	-> [PointerTensor | me:31716758775 -> alice:59855583100]
 	-> [PointerTensor | me:46315943438 -> secure_worker:61191797929]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:95992045306 -> bob:30004191026]
 	-> tensor([3693784181484542318, 2956892147350366108, 3635483576050661328,
         4319787565357710504,  314185413211429184])
 	-> [PointerTensor | me:1912953733 -> secure_worker:74928020820]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:27763109608 -> bob:53768740351]
 	-> [PointerTensor | me:12706007060 -> alice:28519608578]
 	-> tensor([ 366154423538208858, 1701139176182398256, 2571388847450752228,
          993694966516878514, 3658225715673759732])
 	*crypto provider: me*]

In [93]:
x = th.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [0]:
x = x.fix_prec()

In [95]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [96]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [97]:
x = th.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [0]:
y = x + x

In [99]:
y

tensor([0.2000, 0.4000, 0.6000, 0.8000, 1.0000])